<a href="https://colab.research.google.com/github/carlaprv/sin5007-reconhecimento-de-padroes/blob/master/06%20-%20k-fold%20cross%20validation%20%2B%20naive%20Bayes%20v05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Bayes
-------------------------------------------------------------------------------------------------------------

# Bibliotecas Necessárias

In [0]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import seaborn as sns # visualize
import matplotlib.pyplot as plt
import statsmodels.stats.api as sms

from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from statsmodels.stats.weightstats import DescrStatsW



# Funções Auxiliares

describe_dataset() : realiza o cálculo das proporções de classes do dataset original

In [0]:
def describe_dataset(X, y, k):
    # get dataset rows: instances , columns: features
    rows, columns = X.shape
    # get proportion from target
    (unique, counts) = np.unique(y, return_counts=True) 
    # calculate proportion
    prop_neg = int(counts[0]/rows*100)
    prop_pos = int(counts[1]/rows*100)

    print("k = {}, Dataset: {} positivas, {} negativas ({}% x {}%)".format(k, counts[1], counts[0], prop_pos, prop_neg))

get_classes_from_index() : realiza o cálculo das proporções de classes dos folds criados

In [0]:
def get_classes_from_index(y, skf):
    _, y_idx, y_inv = np.unique(y, return_index=True, return_inverse=True)
    y_counts = np.bincount(y_inv)
    _, class_perm = np.unique(y_idx, return_inverse=True)
    y_encoded = class_perm[y_inv]
    y_order = np.sort(y_encoded)
    n_classes = len(y_idx)
    allocation = np.asarray(
            [np.bincount(y_order[i::skf.n_splits], minlength=n_classes)
             for i in range(skf.n_splits)])

    for idx, f in enumerate(allocation):
        count_neg = int(f[0])
        count_pos = int(f[1])
        total = count_neg+count_pos
        prop_temp_neg = int(count_neg/total*100)
        prop_temp_pos = int(count_pos/total*100)
        print("Fold {}: Pos: {}, Neg: {}, Total: {}, Proporção: {}% x {}%".format(idx, count_pos, count_neg, total, prop_temp_pos, prop_temp_neg))

# Função que aplica o Naive Bayes

In [0]:
def stratified_k_fold(X, y, list_c, k, name):
    """
    Parameters
    ----------    
       X : array-like, shape (n_samples, n_features)
           Training data, where n_samples is the number of samples
           and n_features is the number of features.
       y : array-like, of length n_samples
           The target variable for supervised learning problems.
       k : int
           Determines the number of folds.
    name : method selection (string)

    """

    ### Estratifica o dataset em k folds
    skf = StratifiedKFold(n_splits=k)
    describe_dataset(X, y, k)
    get_classes_from_index(y, skf)
    
    
    ### Lista para armazenar os resultados de cada valor de c
    ### Armazena um array bidimensional, onde terá o valor do c e uma lista dos resultados de c
    result = []
    reports_g = []
    
    
    ### Executa o treino e teste para cada valor do parametro c
    for c in list_c:
        print("c =  {}" .format(c))

        ### create naive bayes classifier
        clf = GaussianNB(var_smoothing = c)
        
        
        ### Array para guardar os resultados dos testes para o parametro c
        """
        Coluna 0 : Armazena o valor de c
        Coluna 1 : Armazena o resultado 
        """
        result_c = []

                
        ### resultado do fold-k
        result_k = []
        ### Executa o treino e teste para k folds
        fold_k = 1
        for train_index, test_index in skf.split(X, y):
            
            print("fold_k: {}" .format(fold_k))
            print("\nTRAIN: {}  TEST: {}".format(len(train_index), len(test_index)))
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            ### train classifier
            clf.fit(X_train, y_train)
            
            ### calculate metrics
            y_predicted = clf.predict(X_test)
            report_dict = metrics.classification_report(y_test, y_predicted, output_dict=True)
            report_str = metrics.classification_report(y_test, y_predicted)
                
            ### Armazena o resultado do test do fold-k          
            result_k.append(report_dict)
            print(report_str)
            
            fold_k = fold_k + 1
            

        ### Guarda os resultados dos k fold do parametro c 
        reports_pos = pd.DataFrame(pd.DataFrame(result_k)['1.0'].to_list())
        accuracy_reports = pd.DataFrame(pd.DataFrame(result_k)['accuracy'])
        reports_pos['accuracy'] = accuracy_reports
        reports_pos['pos/neg'] = 'pos'
        reports_neg = pd.DataFrame(pd.DataFrame(result_k)['0.0'].to_list())
        accuracy_reports = pd.DataFrame(pd.DataFrame(result_k)['accuracy'])
        reports_neg['accuracy'] = accuracy_reports
        reports_neg['pos/neg'] = 'neg'
        
        reports = pd.concat([reports_pos,reports_neg])
        print(type(reports))
        print(reports)
        
        ### Guarda o resultado para impressao grafica reports_g
        reports_c = reports
        reports_c['Param(c)'] = c
        reports_c['method'] = str(name)
        reports_g.append(reports_c)
                
        ### Guarda o resultado da execução para o parâmetro c
        result_c = [c, reports]
        result.append(result_c)
        
    
    ### Retorna a lista com todos os resultado para cada c
    return result , reports_g
       

# Função para calcular a média das medidas

In [0]:
# Calcula a média das medidas de cada c
def calcula_media_ic(lista_result,alpha):
    
    mean_c = []
    for result in lista_result:
        
        c = result[0]
        result_c = result[1]
        
        # Calcula a média das medidas do parametro c
        precision_mean = result_c['precision'].mean()
        recall_mean = result_c['recall'].mean()
        f1_score_mean = result_c['f1-score'].mean()
        support_mean = result_c['support'].mean()
        accuracy_mean = result_c['accuracy'].mean()

        # Calcula as ic das medidas
        precision_ic = sms.DescrStatsW(result_c['precision']).tconfint_mean(alpha)
        recall_ic = sms.DescrStatsW(result_c['recall']).tconfint_mean(alpha)
        f1_score_ic = sms.DescrStatsW(result_c['f1-score']).tconfint_mean(alpha)
        support_ic = sms.DescrStatsW(result_c['support']).tconfint_mean(alpha)
        accuracy_ic = sms.DescrStatsW(result_c['accuracy']).tconfint_mean(alpha)

        ic = {'recall_ic' : recall_ic, 'support_ic' : support_ic, 'accuracy_ic': accuracy_ic }
        ic = pd.DataFrame(ic, index=['inf','sup'])

        # Armazena a média das medidas do parametro c
        mean_c.append([c, precision_mean, recall_mean, f1_score_mean, support_mean, accuracy_mean])
    
    name_columns = ['c', 'precision_mean', 'recall_mean', 'f1_score_mean', 'support_mean', 'accuracy_mean']
    mean_c = pd.DataFrame(mean_c, columns=name_columns)
    return mean_c, ic

##### Parâmetros de execução do Naive Bayes
list_c : valores do parâmetro de ajuste de probabilidade 

k_folds : número de folds para a estratificação do dataset

In [0]:
list_c = [0.001, 0.10, 0.25, 0.50, 0.75, 1]
k_folds = 10

# Execução base: Todas as características

In [36]:
df = pd.read_csv('results/dataset-normalizado.csv', header = 0)
X = df.drop('is_approved', axis=1).to_numpy() # DATASET
y = df['is_approved'].to_numpy() # target
result_all_features , result_all_features_g = stratified_k_fold(X, y, list_c, k=k_folds, name='All Features')

k = 10, Dataset: 348 positivas, 301 negativas (53% x 46%)
Fold 0: Pos: 34, Neg: 31, Total: 65, Proporção: 52% x 47%
Fold 1: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 2: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 3: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 4: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 5: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 6: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 7: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 8: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 9: Pos: 34, Neg: 30, Total: 64, Proporção: 53% x 46%
c =  0.001
fold_k: 1

TRAIN: 584  TEST: 65
              precision    recall  f1-score   support

         0.0       0.80      0.13      0.22        31
         1.0       0.55      0.97      0.70        34

    accuracy                           0.57        65
   macro avg       0.68      0.55      0.46        65
weighted avg       0.67      0.57      0.47    

Resultado dos k-fold para cada valor de c

In [37]:
result_all_features

[[0.001,    precision    recall  f1-score  ...  pos/neg  Param(c)        method
  0   0.550000  0.970588  0.702128  ...      pos     0.001  All Features
  1   0.625000  0.857143  0.722892  ...      pos     0.001  All Features
  2   0.763158  0.828571  0.794521  ...      pos     0.001  All Features
  3   0.666667  0.800000  0.727273  ...      pos     0.001  All Features
  4   0.717949  0.800000  0.756757  ...      pos     0.001  All Features
  5   0.775000  0.885714  0.826667  ...      pos     0.001  All Features
  6   0.846154  0.942857  0.891892  ...      pos     0.001  All Features
  7   0.685714  0.685714  0.685714  ...      pos     0.001  All Features
  8   0.888889  0.457143  0.603774  ...      pos     0.001  All Features
  9   0.730769  0.558824  0.633333  ...      pos     0.001  All Features
  0   0.800000  0.129032  0.222222  ...      neg     0.001  All Features
  1   0.705882  0.400000  0.510638  ...      neg     0.001  All Features
  2   0.777778  0.700000  0.736842  ...     

Calcula a média das medidas de cada parâmetro c

In [38]:
result_all_features_mean , result_all_features_ic = calcula_media_ic(result_all_features,0.05)
result_all_features_mean

,c,precision_mean,recall_mean,f1_score_mean,support_mean,accuracy_mean
0,0.001,0.727840,0.700779,0.687743,32.45,0.705625
1,0.100,0.739483,0.710530,0.697445,32.45,0.714880
2,0.250,0.738529,0.706298,0.691531,32.45,0.711779
3,0.500,0.734588,0.698482,0.682769,32.45,0.705625
4,0.750,0.740412,0.702054,0.684977,32.45,0.710240
5,1.000,0.734906,0.695917,0.677640,32.45,0.705625


Obtém as medidas da maior média de acurácia

In [39]:
best_accuracy_all_features = pd.Series(result_all_features_mean.iloc[result_all_features_mean['accuracy_mean'].idxmax()], 
                          name='All Features')
best_all_features = pd.DataFrame(best_accuracy_all_features)
best_all_features

,All Features
c,0.100000
precision_mean,0.739483
recall_mean,0.710530
f1_score_mean,0.697445
support_mean,32.450000
accuracy_mean,0.714880


# Execução Base: PCA

In [40]:
df = pd.read_csv('results/dataset-pca.csv', header = 0)
X = df.drop('is_approved', axis=1).to_numpy() # DATASET
y = df['is_approved'].to_numpy() # target
result_pca , result_pca_g = stratified_k_fold(X, y, list_c, k=k_folds, name='PCA')

k = 10, Dataset: 348 positivas, 301 negativas (53% x 46%)
Fold 0: Pos: 34, Neg: 31, Total: 65, Proporção: 52% x 47%
Fold 1: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 2: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 3: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 4: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 5: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 6: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 7: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 8: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 9: Pos: 34, Neg: 30, Total: 64, Proporção: 53% x 46%
c =  0.001
fold_k: 1

TRAIN: 584  TEST: 65
              precision    recall  f1-score   support

         0.0       1.00      0.16      0.28        31
         1.0       0.57      1.00      0.72        34

    accuracy                           0.60        65
   macro avg       0.78      0.58      0.50        65
weighted avg       0.77      0.60      0.51    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Resultado dos k-fold para cada valor de c

In [41]:
result_pca

[[0.001,
     precision    recall  f1-score  support  accuracy pos/neg  Param(c) method
  0   0.566667  1.000000  0.723404       34  0.600000     pos     0.001    PCA
  1   0.615385  0.914286  0.735632       35  0.646154     pos     0.001    PCA
  2   0.810811  0.857143  0.833333       35  0.815385     pos     0.001    PCA
  3   0.617021  0.828571  0.707317       35  0.630769     pos     0.001    PCA
  4   0.666667  0.685714  0.676056       35  0.646154     pos     0.001    PCA
  5   0.823529  0.800000  0.811594       35  0.800000     pos     0.001    PCA
  6   0.828571  0.828571  0.828571       35  0.815385     pos     0.001    PCA
  7   0.692308  0.514286  0.590164       35  0.615385     pos     0.001    PCA
  8   0.928571  0.371429  0.530612       35  0.646154     pos     0.001    PCA
  9   0.736842  0.411765  0.528302       34  0.609375     pos     0.001    PCA
  0   1.000000  0.161290  0.277778       31  0.600000     neg     0.001    PCA
  1   0.769231  0.333333  0.465116       30

Calcula a média das medidas de cada parâmetro c

In [42]:
result_pca_mean,result_pca_ic = calcula_media_ic(result_pca,alpha=0.05)
result_pca_ic

,recall_ic,support_ic,accuracy_ic
inf,0.515627,31.308895,0.631760
sup,0.807342,33.591105,0.724153


Obtém o resultado da maior média de acurácia 

In [43]:
best_accuracy_pca = pd.Series(result_pca_mean.iloc[result_pca_mean['accuracy_mean'].idxmax()], 
                          name='PCA')
best_pca = pd.DataFrame(best_accuracy_pca)
best_pca

,PCA
c,0.500000
precision_mean,0.739416
recall_mean,0.673532
f1_score_mean,0.643785
support_mean,32.450000
accuracy_mean,0.687187


# Execução Base: Chi Squared (K-Best)

In [44]:
df = pd.read_csv('results/dataset-fs-chi-squared.csv', header = 0)
X = df.drop('is_approved', axis=1).to_numpy() # DATASET
y = df['is_approved'].to_numpy() # target
result_chi , result_chi_g  = stratified_k_fold(X, y, list_c, k=k_folds, name='CHI')

k = 10, Dataset: 348 positivas, 301 negativas (53% x 46%)
Fold 0: Pos: 34, Neg: 31, Total: 65, Proporção: 52% x 47%
Fold 1: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 2: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 3: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 4: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 5: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 6: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 7: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 8: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 9: Pos: 34, Neg: 30, Total: 64, Proporção: 53% x 46%
c =  0.001
fold_k: 1

TRAIN: 584  TEST: 65
              precision    recall  f1-score   support

         0.0       0.83      0.16      0.27        31
         1.0       0.56      0.97      0.71        34

    accuracy                           0.58        65
   macro avg       0.70      0.57      0.49        65
weighted avg       0.69      0.58      0.50    

Resultado dos k-fold para cada valor de c

In [45]:
result_chi

[[0.001,
     precision    recall  f1-score  support  accuracy pos/neg  Param(c) method
  0   0.559322  0.970588  0.709677       34  0.584615     pos     0.001    CHI
  1   0.647059  0.942857  0.767442       35  0.692308     pos     0.001    CHI
  2   0.810811  0.857143  0.833333       35  0.815385     pos     0.001    CHI
  3   0.688889  0.885714  0.775000       35  0.723077     pos     0.001    CHI
  4   0.710526  0.771429  0.739726       35  0.707692     pos     0.001    CHI
  5   0.842105  0.914286  0.876712       35  0.861538     pos     0.001    CHI
  6   0.871795  0.971429  0.918919       35  0.907692     pos     0.001    CHI
  7   0.742857  0.742857  0.742857       35  0.723077     pos     0.001    CHI
  8   0.944444  0.485714  0.641509       35  0.707692     pos     0.001    CHI
  9   0.750000  0.617647  0.677419       34  0.687500     pos     0.001    CHI
  0   0.833333  0.161290  0.270270       31  0.584615     neg     0.001    CHI
  1   0.857143  0.400000  0.545455       30

Calcula a média das medidas de cada parâmetro c

In [46]:
result_chi_mean, result_chi_ic = calcula_media_ic(result_chi,0.05)
result_chi_ic

,recall_ic,support_ic,accuracy_ic
inf,0.589768,31.308895,0.662160
sup,0.827304,33.591105,0.770628


Obtém o resultado da maior média de acurácia

In [47]:
best_accuracy_chi = pd.Series(result_chi_mean.iloc[result_chi_mean['accuracy_mean'].idxmax()], 
                          name='Chi Squared')
best_chi = pd.DataFrame(best_accuracy_chi)
best_chi

,Chi Squared
c,0.001000
precision_mean,0.769488
recall_mean,0.736048
f1_score_mean,0.724163
support_mean,32.450000
accuracy_mean,0.741058


# Execução Base: recursive-feature

In [48]:
df = pd.read_csv('results/dataset-fs-recursive-feature.csv', header = 0)
X = df.drop('is_approved', axis=1).to_numpy() # DATASET
y = df['is_approved'].to_numpy() # target
result_recursive , result_recursive_g  = stratified_k_fold(X, y, list_c, k=k_folds, name='Recursive')

k = 10, Dataset: 348 positivas, 301 negativas (53% x 46%)
Fold 0: Pos: 34, Neg: 31, Total: 65, Proporção: 52% x 47%
Fold 1: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 2: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 3: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 4: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 5: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 6: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 7: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 8: Pos: 35, Neg: 30, Total: 65, Proporção: 53% x 46%
Fold 9: Pos: 34, Neg: 30, Total: 64, Proporção: 53% x 46%
c =  0.001
fold_k: 1

TRAIN: 584  TEST: 65
              precision    recall  f1-score   support

         0.0       1.00      0.13      0.23        31
         1.0       0.56      1.00      0.72        34

    accuracy                           0.58        65
   macro avg       0.78      0.56      0.47        65
weighted avg       0.77      0.58      0.48    

Resultado dos k-fold para cada valor de c 

In [49]:
result_recursive

[[0.001,    precision    recall  f1-score  ...  pos/neg  Param(c)     method
  0   0.557377  1.000000  0.715789  ...      pos     0.001  Recursive
  1   0.640000  0.914286  0.752941  ...      pos     0.001  Recursive
  2   0.761905  0.914286  0.831169  ...      pos     0.001  Recursive
  3   0.636364  0.800000  0.708861  ...      pos     0.001  Recursive
  4   0.727273  0.914286  0.810127  ...      pos     0.001  Recursive
  5   0.761905  0.914286  0.831169  ...      pos     0.001  Recursive
  6   0.850000  0.971429  0.906667  ...      pos     0.001  Recursive
  7   0.644444  0.828571  0.725000  ...      pos     0.001  Recursive
  8   0.857143  0.685714  0.761905  ...      pos     0.001  Recursive
  9   0.750000  0.705882  0.727273  ...      pos     0.001  Recursive
  0   1.000000  0.129032  0.228571  ...      neg     0.001  Recursive
  1   0.800000  0.400000  0.533333  ...      neg     0.001  Recursive
  2   0.869565  0.666667  0.754717  ...      neg     0.001  Recursive
  3   0.66666

Calcula a média das medidas de cada parâmetro c

In [50]:
result_recursive_mean, result_recursive_ic  = calcula_media_ic(result_recursive,0.05)
result_recursive_mean

,c,precision_mean,recall_mean,f1_score_mean,support_mean,accuracy_mean
0,0.001,0.764978,0.722222,0.710783,32.45,0.731875
1,0.100,0.761567,0.717764,0.701076,32.45,0.728774
2,0.250,0.765515,0.711335,0.691684,32.45,0.724183
3,0.500,0.778575,0.701615,0.677929,32.45,0.716466
4,0.750,0.775907,0.694472,0.669918,32.45,0.710288
5,1.000,0.773248,0.685425,0.658916,32.45,0.702572


Obtém o resultado da maior média de acurácia

In [51]:
best_accuracy_recursive = pd.Series(result_recursive_mean.iloc[result_recursive_mean['accuracy_mean'].idxmax()], 
                          name='Recursive Feature')
best_recursive = pd.DataFrame(best_accuracy_recursive)
best_recursive

,Recursive Feature
c,0.001000
precision_mean,0.764978
recall_mean,0.722222
f1_score_mean,0.710783
support_mean,32.450000
accuracy_mean,0.731875


# Junta todos os resultados

In [58]:
result = pd.concat([best_all_features, best_pca, best_chi, best_recursive], axis=1)
print("Média das métricas geradas pelo processamento de cada dataset")
result.transpose()



Média das métricas geradas pelo processamento de cada dataset


,c,precision_mean,recall_mean,f1_score_mean,support_mean,accuracy_mean
All Features,0.100,0.739483,0.710530,0.697445,32.45,0.714880
PCA,0.500,0.739416,0.673532,0.643785,32.45,0.687187
Chi Squared,0.001,0.769488,0.736048,0.724163,32.45,0.741058
Recursive Feature,0.001,0.764978,0.722222,0.710783,32.45,0.731875


In [53]:
result_all_features_ic['Methode'] = 'All_features'
result_pca_ic['Methode'] = 'PCA'
result_chi_ic['Methode'] = 'Chi Squared'
result_recursive_ic['Methode'] = 'Recursive Feature'

ic_s = pd.concat([result_all_features_ic,result_pca_ic,result_chi_ic,result_recursive_ic],axis=0)
print("IC das métricas geradas pelo processamento de cada dataset")
ic_s

IC das métricas geradas pelo processamento de cada dataset


,recall_ic,support_ic,accuracy_ic,Methode
inf,0.576213,31.308895,0.653441,All_features
sup,0.815622,33.591105,0.757809,All_features
inf,0.515627,31.308895,0.631760,PCA
sup,0.807342,33.591105,0.724153,PCA
inf,0.589768,31.308895,0.662160,Chi Squared
sup,0.827304,33.591105,0.770628,Chi Squared
inf,0.542656,31.308895,0.658414,Recursive Feature
sup,0.828194,33.591105,0.746730,Recursive Feature


# Gerar Graficos

In [54]:
frames = result_all_features_g +  result_pca_g + result_chi_g + result_recursive_g

tips = pd.concat(frames)

sns.set(style="whitegrid")

fig, ax = plt.subplots(3,1,figsize=[10,25])

g = sns.barplot(ax=ax[0], x="method", y="accuracy", hue="Param(c)", data=tips,capsize=.05)
g.set(ylim=(0,1),yticks=np.arange(0,1.1,0.1).tolist())
g.set_ylabel("Accuracy",fontsize=30)
g.set_title('Classificador Naive Bayes',fontsize=30)
g = sns.barplot(ax=ax[1], x="method", y="recall", hue="Param(c)", data=tips,capsize=.05);
g.set(ylim=(0,1),yticks=np.arange(0,1.1,0.1).tolist())
g.set_ylabel("Recall",fontsize=30)
g = sns.barplot(ax=ax[2], x="method", y="precision", hue="Param(c)", data=tips,capsize=.05);
g.set(ylim=(0,1),yticks=np.arange(0,1.1,0.1).tolist())
g.set_ylabel("Precision",fontsize=30)
plt.close(2)


